In [1]:
!nvidia-smi

Fri Aug 12 10:19:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# ignoring warnings
import warnings
warnings.simplefilter('ignore')

In [3]:
!pip install -q efficientnet_pytorch

In [4]:
!pip install -U albumentations

In [5]:
!pip install opencv-python-headless==4.2.0.34

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.4.60
    Uninstalling opencv-python-headless-4.5.4.60:
      Successfully uninstalled opencv-python-headless-4.5.4.60


In [6]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.2 MB/s eta 0:00:00


In [7]:
import warnings
warnings.simplefilter('ignore')

import timm
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision import transforms
import torch
import torchvision
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import train_test_split

#Keras 
from keras.utils.np_utils import to_categorical

In [8]:
import random
SEED_VAL  = 15
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
    random.seed(SEED_VAL)
    np.random.seed(SEED_VAL)
    torch.manual_seed(SEED_VAL)
    torch.cuda.manual_seed_all(SEED_VAL)
    os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [9]:
import cv2
def default_image_loader(path):
    return Image.open(path).convert('RGB')

def cv_reader(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

class ImageDataset(Dataset):
    def __init__(self, data_path, df, transform=None,mode='train'):
        self.df = df
        self.loader = cv_reader
        self.transform = transform
        self.dir = data_path
        self.mode = mode

    def __getitem__(self, index):
        image_name = self.df.image_name[index]
        image = self.loader(os.path.join(self.dir, image_name))
        if self.transform is not None:
          image = self.transform(image=image)
          image=image["image"]

          output = {
              'image': image,
          }

        if self.mode == 'train':
            label = self.df.loc[index, 'target']
            output.update({
              'label': torch.tensor(label, dtype=torch.long)
          })

        return output          
        
    
    def __len__(self):
        return self.df.shape[0]

In [10]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, name):
        super(Net, self).__init__()
        self.name  = name
        if name == 'b0':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b0')
          self.arch._fc = nn.Linear(in_features=1280, out_features=3, bias=True)
      
        elif name == 'swin_base_patch4_window12_384_in22k':
          self.arch = timm.create_model("swin_base_patch4_window12_384_in22k", pretrained=True,  num_classes=3)
        
        elif name == 'swin_large_patch4_window12_384_in22k':
          self.arch = timm.create_model("swin_large_patch4_window12_384_in22k", pretrained=True,  num_classes=3)
#           num_features = self.arch.head.in_features
#           self.arch.head = nn.Linear(num_features, 128)
#           self.dropout = nn.Dropout(p=0.15)
#           self.fully_connect = nn.Sequential(nn.Linear(128, 64),
#                                                nn.ReLU(),
#                                                nn.Linear(64, 3)
#                                               )
      
    def forward(self, x):
        x = self.arch(x)
#         x = self.dropout(x)
#         output = self.fully_connect(x)
        return x

In [11]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [14]:
def loss_fn(outputs,targets):
  criterion = nn.CrossEntropyLoss()
  loss = criterion(outputs,targets)
  return loss

In [15]:
def train_fn(train_data_loader, model, optimizer, device, scheduler=None):
  avg_acc=0
  model.to(device)
  model.train()
  losses = AverageMeter()
  tk0 = tqdm(train_data_loader, total=len(train_data_loader))
  tot_loss = 0
  for bi,d in enumerate(tk0):
    images = d['image']
    labels = d['label']

    #send them to device 
    images = images.to(device, dtype=torch.float)
    labels = labels.to(device)
    optimizer.zero_grad()
    if BOOL_CROP:
      bs, ncrops, c, h, w = images.size()
      outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
      outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
    
    else:
      outputs  = model(images)

    loss = loss_fn(outputs, labels)
    outputs = F.softmax(outputs, dim=1)
#     tot_loss += loss
    tot_loss = tot_loss + loss.item()
    avg_acc += (labels == outputs.argmax(1)).float().mean()
#     losses.update(loss.item(), labels.size(0))
    
    loss.backward()
    optimizer.step()
    


    if scheduler is not None:
      scheduler.step()

  loglosss = tot_loss/len(train_data_loader)
  avg_acc_nump = avg_acc.cpu().detach().numpy()
  acc_avg = avg_acc_nump/len(train_data_loader)
  
  print("Training loss for this epoch: ", loglosss)
  return loglosss

In [16]:
def eval_fn(valid_data_loader,model,device):
  model.eval()
  losses = AverageMeter()
  tot_loss = 0
  final_outputs = []
  final_targets = []
  probabilities = []
  with torch.no_grad():
    for bi,d in enumerate(valid_data_loader):

      images = d['image']
      labels = d['label']
      

      #send them to device 
      images = images.to(device,dtype=torch.float)
      labels = labels.to(device)
      if BOOL_CROP:
        bs, ncrops, c, h, w = images.size()
        outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
        outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
      else:
        outputs  = model(images)
      probabilities.append(F.softmax(outputs, dim=-1).cpu().detach().numpy())
      loss = loss_fn(outputs,labels)
      outputs = F.softmax(outputs, dim=1)
#       tot_loss += loss
      tot_loss = tot_loss + loss.item()
      losses.update(loss.item(), labels.size(0))

      
      final_outputs.append(outputs.cpu().detach().numpy())
      final_targets.append(labels.cpu().numpy())


    final_targets = np.concatenate(final_targets)
    final_outputs = np.concatenate(final_outputs)
    probabilities = np.concatenate(probabilities)
    loglosss  = tot_loss/len(valid_data_loader)
    print("Validation loss for this epoch: ",loglosss)
  return loglosss,probabilities

In [17]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
from albumentations.pytorch.transforms import ToTensorV2

input_shape = [128,256]

def get_transforms():
    train_transform = A.Compose([
                              A.Resize(384, 384, p=1),
                              A.RandomResizedCrop(height=384, width=384, p=1),
                              A.Flip(p=0.5),
                              A.HorizontalFlip(p=0.5),
                              A.VerticalFlip(p=0.5),
                              A.Rotate(15),
                              A.ShiftScaleRotate(p=0.5),
                              ##A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
                              #A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),  
#                               A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, always_apply=False, p=0.5)
                              #A.CoarseDropout(p=0.5),
                              #A.Cutout(p=0.5),
                              A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                              ToTensorV2(),
                            ])
    val_transform = A.Compose([
                              A.Resize(height=384, width=384, p=1),
                              A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                              ToTensorV2(),
                            ])
    return train_transform,val_transform

In [18]:
train = pd.read_csv('../input/csv-files-for-microsoft-rice-disease/Train.csv')
train = train.rename(columns = {'Image_id':'image_name','Label':'target'})

In [19]:
train

,image_name,target
0,id_004wknd7qd.jpg,blast
1,id_004wknd7qd_rgn.jpg,blast
2,id_005sitfgr2.jpg,brown
3,id_005sitfgr2_rgn.jpg,brown
4,id_00stp9t6m6.jpg,blast
...,...,...
5335,id_zz6gzk7p97_rgn.jpg,brown
5336,id_zz8ca2p67e.jpg,blast
5337,id_zz8ca2p67e_rgn.jpg,blast
5338,id_zzt8y9q0x0.jpg,brown


In [20]:
#identify partial string to look for
discard = ["_rgn"]

#drop rows that contain the partial string "_rgn" in the conference column
train = train[~train.image_name.str.contains('|'.join(discard))]

In [21]:
train.reset_index(drop=True, inplace=True)

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'])

In [23]:
dict(zip(le.classes_, le.transform(le.classes_)))

{'blast': 0, 'brown': 1, 'healthy': 2}

In [24]:
%%time
from sklearn.model_selection import StratifiedKFold, KFold
NAME = 'swin_large_patch4_window12_384_in22k'
EPOCHS = 15
TRAIN_BATH_SIZE = 4
BOOL_CROP = False
skf = StratifiedKFold(n_splits=5)
all_loss_scores = []
def run_folds():
  seed_all(SEED_VAL)
  train_transform,val_transform = get_transforms()
  DF = pd.DataFrame()
  oof = np.zeros((train.shape[0],3))
  for i,(train_index,val_index) in enumerate(skf.split(train, y=train.target)):
        print(f"#########################  Fold {i+1}/{skf.n_splits}  #########################")
        train_df , valid_df = train.iloc[train_index,:],train.iloc[val_index,:]
        train_df = train_df.reset_index()
        valid_df = valid_df.reset_index()
        train_dataset = ImageDataset('../input/microsoft-rice-disease-classification-challenge', train_df,train_transform)
        valid_dataset = ImageDataset('../input/microsoft-rice-disease-classification-challenge',valid_df,val_transform)
        train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=TRAIN_BATH_SIZE)
        valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False, batch_size=4)
        device = torch.device("cuda")
        model = Net(NAME)
        model.to(device)

        num_train_steps = int(len(train_df) /TRAIN_BATH_SIZE * EPOCHS)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
        best_loss = 3.0

        for epoch in range(EPOCHS):
          print("----------------EPOCH "+str(epoch+1)+"---------------------")
          loss_train = train_fn(train_data_loader, model, optimizer, device,scheduler=None)
          loss_val,probabilities = eval_fn(valid_data_loader ,model, device)
          if loss_val<best_loss:
            best_loss = loss_val
            oof[val_index]=probabilities
            torch.save(model.state_dict(),f"best_model_{i}") 
        print(f'best VAL_LOSS for fold {i+1}: ',best_loss)
        DF = DF.append({'FOLD':1+i,'BEST_LOG_LOSS':best_loss},ignore_index=True)
        all_loss_scores.append(best_loss)
  print(f"MEAN LogLoss: {np.mean(all_loss_scores)}")
  return DF,oof

CPU times: user 0 ns, sys: 26 µs, total: 26 µs
Wall time: 29.8 µs


In [25]:
%%time
kfold_results,oof = run_folds()

#########################  Fold 1/5  #########################


Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window12_384_22k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window12_384_22k.pth


----------------EPOCH 1---------------------


  0%|          | 0/534 [00:00<?, ?it/s]

Training loss for this epoch:  0.515178336524701
Validation loss for this epoch:  0.21924586314707994
----------------EPOCH 2---------------------


...

Training loss for this epoch:  0.06885931729995944
Validation loss for this epoch:  0.09285853259634949
----------------EPOCH 14---------------------


  0%|          | 0/534 [00:00<?, ?it/s]

Training loss for this epoch:  0.06064902777454128
Validation loss for this epoch:  0.08234639714090401
----------------EPOCH 15---------------------


  0%|          | 0/534 [00:00<?, ?it/s]

Training loss for this epoch:  0.046349240346367206
Validation loss for this epoch:  0.09343194225766213
best VAL_LOSS for fold 5:  0.08149586065706194
MEAN LogLoss: 0.0693885794026925
CPU times: user 6h 54min 1s, sys: 3min 6s, total: 6h 57min 8s
Wall time: 6h 56min 19s


In [26]:
from sklearn.metrics import log_loss
log_loss(train['target'].values,oof)

0.06944918654528237

In [27]:
OOF_csv = train[["image_name","target"]]
OOF_csv['target'] = OOF_csv['target'].apply(lambda x :le.inverse_transform([x])[0])
OOF_csv['blast'] = oof[:,0]
OOF_csv['brown'] = oof[:,1]
OOF_csv['healthy'] = oof[:,2]
OOF_csv.to_csv("OOF for caleb notebook new aug.csv",index=False)

In [28]:
OOF_csv

,image_name,target,blast,brown,healthy
0,id_004wknd7qd.jpg,blast,0.999638,0.000307,0.000055
1,id_005sitfgr2.jpg,brown,0.000048,0.999943,0.000008
2,id_00stp9t6m6.jpg,blast,0.971935,0.000045,0.028020
3,id_012zxewnhx.jpg,blast,0.998585,0.000038,0.001376
4,id_0186qwq2at.jpg,healthy,0.257140,0.001186,0.741673
...,...,...,...,...,...
2665,id_zydzdp046u.jpg,healthy,0.000858,0.002783,0.996359
2666,id_zyoowbqcm3.jpg,healthy,0.000484,0.001656,0.997860
2667,id_zz6gzk7p97.jpg,brown,0.999842,0.000156,0.000002
2668,id_zz8ca2p67e.jpg,blast,0.999997,0.000001,0.000002


In [29]:
kfold_results

,FOLD,BEST_LOG_LOSS
0,1.0,0.058059
1,2.0,0.037226
2,3.0,0.104935
3,4.0,0.065228
4,5.0,0.081496


In [30]:
test_transform = A.Compose([        
                              A.Resize(height=384, width=384,p=1),
                              A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                              ToTensorV2()])
test = pd.read_csv('../input/csv-files-for-microsoft-rice-disease/Test.csv')
test = test[~test.Image_id.str.contains('_rgn')]
test.reset_index(inplace=True,drop=True)
test.rename(columns={'Image_id':'image_name'},inplace=True)
test_dataset = ImageDataset('../input/microsoft-rice-disease-classification-challenge',test, test_transform, mode='test')

In [31]:
def predict(bs=4):
  test_ds = ImageDataset('./Images',test, test_transform,mode='test')
  testloader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=bs)

  predictions_labels = []
  predictions_proba = []

  out = None

  for data in tqdm(testloader):
    x = data['image'].to(device, dtype=torch.float)

    for i in range(skf.n_splits):
      if i == 0: out = F.softmax(MODELS[i](x), dim=-1)
      else: out += F.softmax(MODELS[i](x), dim=-1)

    out /= skf.n_splits
    out_labels = out.argmax(1).cpu().detach().numpy()
    out_probas = out.cpu().detach().numpy()

    predictions_labels += out_labels.tolist()
    predictions_proba += out_probas.tolist()

  return predictions_labels ,predictions_proba

In [32]:
MODELS = []
for i in range(skf.n_splits):
  MODELS.append(Net(name=NAME))
  MODELS[i].to(device)
  MODELS[i].load_state_dict(torch.load(f'best_model_{i}'))
  MODELS[i].eval()

In [33]:
torch.cuda.empty_cache()

In [34]:
predictions_labels, predictions_proba = predict(bs=1)

  0%|          | 0/1145 [00:00<?, ?it/s]

In [35]:
np.save('y_pred_tot.npy', predictions_proba)

In [36]:
submission = pd.read_csv("../input/csv-files-for-microsoft-rice-disease/Test.csv")[['Image_id']]
submission = submission[~submission.Image_id.str.contains('_rgn')]
submission.reset_index(inplace=True,drop=True)

In [37]:
submission

,Image_id
0,id_00vl5wvxq3.jpg
1,id_01hu05mtch.jpg
2,id_030ln10ewn.jpg
3,id_03z57m8xht.jpg
4,id_04ngep1w4b.jpg
...,...
1140,id_zrdlgjrq3r.jpg
1141,id_zsfayxwipp.jpg
1142,id_ztvp2l9k3h.jpg
1143,id_zwwcma7hlt.jpg


In [38]:
submission['blast'] = [p[0] for p in predictions_proba]
submission['brown'] = [p[1] for p in predictions_proba]
submission['healthy'] = [p[2] for p in predictions_proba]

In [39]:
submission.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999998,1.437548e-06,5.914155e-07
1,id_01hu05mtch.jpg,0.028532,9.713973e-01,7.107489e-05
2,id_030ln10ewn.jpg,0.834670,1.649136e-01,4.168361e-04
3,id_03z57m8xht.jpg,0.999999,8.962439e-07,1.104109e-07
4,id_04ngep1w4b.jpg,0.998832,1.007395e-03,1.610138e-04


In [40]:
submission.to_csv('rice_cv_0.06.csv', index=False)